# News Titles Sentiment Analysis

Useful Links:

https://www.datacareer.de/blog/accessing-the-news-api-in-python/

https://newsapi.org/docs/client-libraries/python

https://newsapi.org/docs/get-started

In [77]:
import pprint
import requests
import pandas as pd

In [78]:
secret_api = '4506abb974f8420e997c5cfa21863b17'


In [79]:
url = 'https://newsapi.org/v2/everything?'


In [89]:
'''
from newsapi import NewsApiClient

newsapi = NewsApiClient(api_key='4506abb974f8420e997c5cfa21863b17')

all_articles = newsapi.get_everything(q='FinTech',
                                      #sources='bbc-news,the-verge',
                                      #domains='bbc.co.uk,techcrunch.com',
                                      from_param='2020-05-28',
                                      to='2020-06-27',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

all_articles
'''

"\nfrom newsapi import NewsApiClient\n\nnewsapi = NewsApiClient(api_key='4506abb974f8420e997c5cfa21863b17')\n\nall_articles = newsapi.get_everything(q='FinTech',\n                                      #sources='bbc-news,the-verge',\n                                      #domains='bbc.co.uk,techcrunch.com',\n                                      from_param='2020-05-28',\n                                      to='2020-06-27',\n                                      language='en',\n                                      sort_by='relevancy',\n                                      page=2)\n\nall_articles\n"

In [87]:
## get articles from last month: 

parameters = {
    'q': 'FinTech', # query phrase
    'pageSize': 100,  # maximum is 100
    'apiKey': secret_api # your own API key

}

In [88]:
# Make the request
response = requests.get(url, params=parameters)

# Convert the response to JSON format and pretty print it
response_json = response.json()
pprint.pprint(response_json)

{'articles': [{'author': 'Jonathan Shieber',
               'content': 'This could have been Marqeta’s year to list as a '
                          'public company on a major American stock '
                          'exchange.\r\n'
                          'The company, while still unprofitable, is a darling '
                          'of the financial services sector and only last… '
                          '[+4297 chars]',
               'description': 'This could have been Marqeta’s year to list as '
                              'a public company on a major American stock '
                              'exchange. The company, while still '
                              'unprofitable, is a darling of the financial '
                              'services sector and only last year reached a $2 '
                              'billion valuation on the back of a $260 mi…',
               'publishedAt': '2020-05-28T11:30:52Z',
               'source': {'id': 'techcrunch', 'name

               'publishedAt': '2020-06-15T18:08:51Z',
               'source': {'id': None, 'name': 'Genbeta.com'},
               'title': 'La startup española de pagos Verse, adquirida por '
                        "Square, la compañía 'fintech' del CEO de Twitter",
               'url': 'https://www.genbeta.com/actualidad/startup-espanola-pagos-verse-adquirida-square-compania-fintech-ceo-twitter',
               'urlToImage': 'https://i.blogs.es/a7bec2/suaqre_verse/840_560.jpg'},
              {'author': 'Babu Mohan',
               'content': 'Samsung today introduced a mobile-first money '
                          'management experience called Samsung Money by SoFi. '
                          'It brings a Mastercard debit card and a cash '
                          'management account with exclusive Samsung Pay '
                          'benefits, … [+2376 chars]',
               'description': 'It is set to launch later this summer.\n'
                              '\n'
     

In [67]:
testDataSet = []
for i in response_json['articles']:
    testDataSet.append({'text':i['title'], 'label':None})

In [68]:
testDataSet[0:2]

[{'text': 'Russia and Turkey risk turning Libya into another Syria',
  'label': None},
 {'text': 'Last JFK sibling and envoy Jean Kennedy Smith dies', 'label': None}]

We will now import the training set we have used for the twitter sentiment analysis:

In [69]:
trainingData_copied = pd.read_csv("tweetDataFile.csv", header = None, names = ['tweet_id', 'text', 'label', 'topic'])

df = trainingData_copied.copy()
lst_labels = df['label'].unique()
count_rows_keep = df['label'].value_counts().min()

neutral_df = df[df['label'] == 'neutral'].sample(n= count_rows_keep , random_state = 1)
irrelevant_df = df[df['label'] == 'irrelevant'].sample(n= count_rows_keep , random_state = 2)
negative_df = df[df['label'] == 'negative'].sample(n= count_rows_keep , random_state = 3)
positive_df = df[df['label'] == 'positive'].sample(n= count_rows_keep , random_state = 2)

lst_df = [neutral_df, irrelevant_df, negative_df, positive_df]

trainingData_copied = pd.concat(lst_df)
trainingData_copied['label'].value_counts()

trainingData_copied = trainingData_copied.to_dict('records')
trainingData_copied[0:2]

[{'tweet_id': 125828984293425152,
  'text': 'ok it is back @iphone @apple',
  'label': 'neutral',
  'topic': 'apple'},
 {'tweet_id': 126745275686600705,
  'text': '@WSPSanDiego Kickoff tomorrow 10/20 at San Diego #Microsoft Store at 6:00pm @wspwest http://t.co/5FiP2vVg or ping me #in',
  'label': 'neutral',
  'topic': 'microsoft'}]

Just like we have done during for twitter sentiment analysis, we will pre-process the test and training sets using NLTK library:

In [70]:
import re #a library that makes parsing strings and modifying them more efficient
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk #Natural Processing Toolkit that takes care of any processing that we need to perform on text 
            #to change its form or extract certain components from it.
    
#nltk.download('popular') #We need this if certain nltk libraries are not installed. 

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

In [71]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData_copied)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

## Use the Naive Bayes Classifier

In [94]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
    print("Number of negative comments = " + str(NBResultLabels.count('negative')))
    print("Number of positive comments = " + str(NBResultLabels.count('positive')))
    print("Number of neutral comments = " + str(NBResultLabels.count('neutral')))
    print("Number of irrelevant comments = " + str(NBResultLabels.count('irrelevant')))
    
import plotly.graph_objects as go

sentiment = ["Negative","Positive","Neutral","Irrelevant" ]

fig = go.Figure([go.Bar(x=sentiment, y=[str(NBResultLabels.count('negative')), str(NBResultLabels.count('positive')), str(NBResultLabels.count('neutral')),  str(NBResultLabels.count('irrelevant'))])])
    
fig.update_layout(template = 'simple_white',
    title_text='News Sentiment Results', 
    yaxis=dict(
    title='Percentage (%)',
    titlefont_size=16,
    tickfont_size=14,) ,
        
        
)



fig.show()

Overall Negative Sentiment
Negative Sentiment Percentage = 26.0%
Positive Sentiment Percentage = 12.0%
Number of negative comments = 26
Number of positive comments = 12
Number of neutral comments = 62
Number of irrelevant comments = 0


In [49]:
# add explanations